# Dependencies

In [9]:
###Load in the required packages 

#for linear algebra
import numpy as np

#for data manipulation
import pandas as pd

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")

In [10]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')
train.drop('Unnamed: 0', axis = 1,inplace = True)

# Load data

In [11]:
train.head()

,id,high_BP,heart_condition_detected_2017,married,average_blood_sugar,BMI,TreatmentA,TreatmentB,TreatmentC,TreatmentD,...,Quit,NonSmoker,ActiveSmoker,Job: Gov,Job: Pvt,Job: Bus,Job: Unemp,Job: Parental,Loc: City,Loc: Remote
0,16053,1.0,0.0,1.0,71.67,36.6,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1459,0.0,0.0,0.0,107.95,30.4,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,7678,1.0,0.0,1.0,76.49,42.1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,34943,0.0,0.0,1.0,113.98,57.3,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,17741,0.0,0.0,0.0,70.60,26.7,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
test.head()

,id,high_BP,heart_condition_detected_2017,married,average_blood_sugar,BMI,TreatmentA,TreatmentB,TreatmentC,TreatmentD,...,Quit,NonSmoker,ActiveSmoker,Job: Gov,Job: Pvt,Job: Bus,Job: Unemp,Job: Parental,Loc: City,Loc: Remote
0,33327,0.0,0.0,1.0,76.05,33.4,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,839,0.0,0.0,1.0,73.77,30.1,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,11127,0.0,0.0,1.0,62.95,30.8,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,20768,0.0,0.0,1.0,68.81,36.5,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,37774,0.0,0.0,0.0,122.89,30.8,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
train =train[~train.isin([np.nan, np.inf, -np.inf]).any(1)]
test =test[~test.isin([np.nan, np.inf, -np.inf]).any(1)]

# Define Submission Function

In [215]:
def submit(predictions, threshold = 0.05):
    
    # Create submission table
    submission = test[['id']]
    submission['stroke_in_2018'] = predictions
    
    # Where set predictions equal to 0 or 1 based on threshold
    submission['stroke_in_2018'] = np.where(submission['stroke_in_2018'] > threshold, 1, 0)
    
    # Define submission as a global variable so you can get details
    globals()['submission'] = submission
    
    # Save
    submission.to_csv('submission.csv', index = False)
    
    # Print the head of the submission
    return submission.head(10)

# Basic Models
Basic ML sklearn classifiers. These sucked.

In [96]:
y_train = train[['stroke_in_2018']]
x_train = train.drop('stroke_in_2018', axis = 1)

In [97]:
y_train.head()    

,stroke_in_2018
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [109]:
#import libraries
from sklearn import tree
from sklearn.naive_bayes import GaussianNB as GNB

# create a variable to store the decision tree model, then train it on the data set above
clf_tree = tree.DecisionTreeClassifier().fit(x_train,y_train)
clf_gnb = GNB().fit(x_train,y_train)

# and one to predict for a given x observed
yhat_tree = clf_tree.predict(test)
yhat_gnb = clf_gnb.predict(test)

<class 'numpy.ndarray'>


In [112]:
submit(yhat_tree, 0.03)

,id,stroke_in_2018
0,33327,0.0
1,839,0.0
2,11127,0.0
3,20768,0.0
4,37774,0.0
5,4283,0.0
6,13832,0.0
7,4579,0.0
8,26781,0.0
9,17196,0.0


In [102]:
submission.describe()

,id,stroke_in_2018
count,8718.000000,8718.000000
mean,21711.645102,0.021335
std,12653.568917,0.144507
min,0.000000,0.000000
25%,10741.500000,0.000000
50%,21764.500000,0.000000
75%,32644.750000,0.000000
max,43580.000000,1.000000


In [103]:
submit(yhat_gnb, 0.05)

,id,stroke_in_2018
0,33327,0.0
1,839,0.0
2,11127,0.0
3,20768,0.0
4,37774,0.0
5,4283,0.0
6,13832,0.0
7,4579,0.0
8,26781,0.0
9,17196,0.0


In [104]:
submission.describe()

,id,stroke_in_2018
count,8718.000000,8718.000000
mean,21711.645102,0.027644
std,12653.568917,0.163960
min,0.000000,0.000000
25%,10741.500000,0.000000
50%,21764.500000,0.000000
75%,32644.750000,0.000000
max,43580.000000,1.000000


# Random Forest
The most successful early classifier. The model reached its limit probably due to non iterative learning. Hyper-parameter tuning was not an option for this hackathon, but we could have tried better feature engineering. Interactions, particularly between treatments, would've enhanced our significant predictors.

In [105]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

# Get feature importance
features = list(x_train.columns)
feature_importance_values = np.zeros(len(features))

In [131]:
# Train on the training data

random_forest.fit(x_train, y_train)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]

print(type(predictions))

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


<class 'numpy.ndarray'>


In [132]:
submit(predictions, 0.03)

,id,stroke_in_2018
0,33327,0
1,839,0
2,11127,0
3,20768,0
4,37774,0
5,4283,1
6,13832,0
7,4579,0
8,26781,1
9,17196,0


In [134]:
submission.describe()

,id,stroke_in_2018
count,8718.000000,8718.000000
mean,21711.645102,0.158064
std,12653.568917,0.364822
min,0.000000,0.000000
25%,10741.500000,0.000000
50%,21764.500000,0.000000
75%,32644.750000,0.000000
max,43580.000000,1.000000


# Symmetric Training
An attempt to overcome a bias target variable by resampling. This didn't work as well as I'd hoped - leading to results marginally better than simple models like logistic regression. It also had two parameters to play with - the amount of resampling in the training data and the threshold of the classifier.

For future projects we learned to use a SMOTE.

In [170]:
# Take random sample with replacement of training observations with strokes
stroke = train[train['stroke_in_2018'] == 1]
stroke_samp = stroke.iloc[list(np.random.choice(np.arange(len(stroke)), 5000)),]

#... and again for no strokes, without replacement
NoStroke = train[train['stroke_in_2018'] == 0]
NoStroke_trainindex = list(np.random.choice(np.arange(len(NoStroke)), 27500, replace = False))
NoStroke_samp = NoStroke.iloc[NoStroke_trainindex,]

# The NoStrokes that aren't used in training will be used as a validation set
NoStroke_testindex = list(set(range(len(NoStroke)))^set(NoStroke_trainindex))
NoStroke_testsamp = NoStroke.iloc[NoStroke_testindex,]

In [171]:
# The training dataframe is made of 5000 Stroke and 27500 Nostroke samples
SYMdf = stroke_samp.append(NoStroke_samp)
SYMdf['stroke_in_2018'].value_counts()

0.0    27500
1.0     5000
Name: stroke_in_2018, dtype: int64

In [200]:
# The testing dataframe is made of the remaining NoStroke and the resampled Stroke observations
SYMdf_test = NoStroke_testsamp.append(stroke)
SYMdf_test['stroke_in_2018'].value_counts()
y_SYMdf_test = SYMdf_test['stroke_in_2018']
x_SYMdf_test = SYMdf_test.drop('stroke_in_2018', axis = 1)
SYMdf_test_target.value_counts()

0.0    6720
1.0     652
Name: stroke_in_2018, dtype: int64

In [206]:
# Make the random forest classifier

from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

y_SYMtrain = SYMdf[['stroke_in_2018']]
x_SYMtrain = SYMdf.drop('stroke_in_2018', axis = 1)
SYMfeatures = list(x_SYMtrain.columns)
feature_importance_values = np.zeros(len(SYMfeatures))

In [207]:
# Train on the training data

random_forest.fit(x_SYMtrain, y_SYMtrain)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': SYMfeatures, 'importance': feature_importance_values})

# Evaluate the model by making a prediction on the rest of the training data
predictions = random_forest.predict_proba(x_SYMdf_test)[:, 1]
predictions[:10]

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


array([0.14, 0.  , 0.01, 0.01, 0.03, 0.  , 0.  , 0.01, 0.02, 0.05])

In [208]:
#Create a submission dataframe for validation

#Actual values
y_SYMdf_test = pd.DataFrame(y_SYMdf_test)
y_SYMdf_test = y_SYMdf_test.reset_index(drop = True)

# Predicted values
predictions = pd.DataFrame(predictions, columns = ['Prob'])
CV = pd.concat([y_SYMdf_test['stroke_in_2018'], predictions], axis = 1)
CV.head(10)

,stroke_in_2018,Prob
0,0.0,0.14
1,0.0,0.00
2,0.0,0.01
3,0.0,0.01
4,0.0,0.03
5,0.0,0.00
6,0.0,0.00
7,0.0,0.01
8,0.0,0.02
9,0.0,0.05


In [209]:
# Create this loop to determine best threshold

for thresh in range(13,17):
    # predictions
    colname = 'Pred ' + str(thresh/100)
    CV[ colname ] = 0
    
    # Where the predicted probability of stroke is higher than the threshold, set = 1
    CV[colname][CV['Prob'] > thresh/100] = 1
    
    # Repeat for matches
    colname2 = 'Match ' + str(thresh/100)
    CV[colname2] = 0
    
    # Set =1 where prediction was correct
    CV[colname2][CV['stroke_in_2018'] == CV[colname]] = 1

In [210]:
# Mean row tells the percentage which had the highest training accuracy, and the percentage of strokes we predict
CV.describe()

,stroke_in_2018,Prob,Pred 0.13,Match 0.13,Pred 0.14,Match 0.14,Pred 0.15,Match 0.15,Pred 0.16,Match 0.16
count,7372.000000,7372.000000,7372.000000,7372.000000,7372.000000,7372.000000,7372.000000,7372.000000,7372.000000,7372.000000
mean,0.088443,0.116875,0.147857,0.940315,0.140260,0.947911,0.134699,0.953473,0.130494,0.957678
std,0.283957,0.280183,0.354982,0.236919,0.347281,0.222222,0.341425,0.210639,0.336869,0.201337
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,0.000000,0.060000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


It seems that the higher the threshold, the better more accurate the prediction. This indicates the symmetrical resampling didn't work - submission results ended up very different to the 

In [211]:
# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]
predictions[(predictions>0.5)]

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


array([0.51, 0.71, 0.58, 0.64, 0.64, 0.81, 0.55, 0.54, 0.54, 0.58, 0.57,
       0.51, 0.6 ])

In [216]:
submit(predictions, 0.05)

,id,stroke_in_2018
0,33327,1
1,839,0
2,11127,0
3,20768,0
4,37774,0
5,4283,0
6,13832,1
7,4579,0
8,26781,1
9,17196,1


In [218]:
submission.describe()

,id,stroke_in_2018
count,8718.000000,8718.000000
mean,21711.645102,0.178252
std,12653.568917,0.382747
min,0.000000,0.000000
25%,10741.500000,0.000000
50%,21764.500000,0.000000
75%,32644.750000,0.000000
max,43580.000000,1.000000


# Interactions
Interactions were determined intuitively. Treatments are used together and so we should look at their interactions

In [219]:
def Interact(df,var1,var2):
    a = df[var1]*df[var2]
    return a

In [220]:
     
train['high_BP_BMI'] =  Interact(train,'BMI','high_BP') 
train['high_BP_BMI'].head()
test['high_BP_BMI'] =  Interact(test,'BMI','high_BP') 
test['high_BP_BMI'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: high_BP_BMI, dtype: float64

In [221]:
train['Age_HeartCondDet'] =  Interact(train,'Age','heart_condition_detected_2017') 
train['Age_HeartCondDet'].head()

test['Age_HeartCondDet'] =  Interact(test,'Age','heart_condition_detected_2017') 
test['Age_HeartCondDet'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Age_HeartCondDet, dtype: float64

In [222]:
interacting = ['TreatmentA', 'TreatmentB', 'TreatmentC','TreatmentD']

for i in range(len(interacting)):
    j = i+1
    var1 = interacting[i]
    while j <= 3:
        var2 = interacting[j]
        test[var1+'_'+var2] =  Interact(test,var1,var2) 
        j +=1

In [223]:
test.columns

Index(['id', 'high_BP', 'heart_condition_detected_2017', 'married',
       'average_blood_sugar', 'BMI', 'TreatmentA', 'TreatmentB', 'TreatmentC',
       'TreatmentD', 'Male', 'Female', 'Other', 'Age', 'Quit', 'NonSmoker',
       'ActiveSmoker', 'Job: Gov', 'Job: Pvt', 'Job: Bus', 'Job: Unemp',
       'Job: Parental', 'Loc: City', 'Loc: Remote', 'high_BP_BMI',
       'Age_HeartCondDet', 'TreatmentA_TreatmentB', 'TreatmentA_TreatmentC',
       'TreatmentA_TreatmentD', 'TreatmentB_TreatmentC',
       'TreatmentB_TreatmentD', 'TreatmentC_TreatmentD'],
      dtype='object')

# Light Gradient Boosting
The most successful gradient boosting tool we used. Used iterative learning, gradient descent, and boosted random forests to get progressively better. Issues with threshold in the end, as we expected training and testing data to have similar amounts of strokes. This was not the case, adding to the relative innaccuracies of all the participants models.

In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the stroke_in_2018 column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `id` and `stroke_in_2018` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['id']
    test_ids = test_features['id']
    
    # Extract the labels for training
    labels = features['stroke_in_2018']
    
    # Remove the ids and target
    features = features.drop(columns = ['id', 'stroke_in_2018'])
    test_features = test_features.drop(columns = ['id'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()

    # Make the submission dataframe
    submission = pd.DataFrame({'id': test_ids, 'stroke_in_2018': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

ModuleNotFoundError: No module named 'lightgbm'

In [225]:
submission, fi, metrics = model(train, test)
print('Baseline metrics')
print(metrics)

NameError: name 'model' is not defined

In [0]:
submission['stroke_in_2018'][submission['stroke_in_2018'] > 0.22] = 1
submission['stroke_in_2018'][submission['stroke_in_2018'] <= 0.22] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
submission.describe()

,id,stroke_in_2018
count,8718.000000,8718.000000
mean,21711.645102,0.434045
std,12653.568917,0.495659
min,0.000000,0.000000
25%,10741.500000,0.000000
50%,21764.500000,0.000000
75%,32644.750000,1.000000
max,43580.000000,1.000000


In [0]:
submission.to_csv('submission.csv', index = False)

# Review
Data cleaning went well
Feature engineering should've been done sooner, and we should've done selection
Cut straight to the better models after preprocessing - XGBoost and neural nets, now you know how to use them